In [2]:
# Dependencies
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import pandas as pd
import tweepy

In [3]:
browser = Browser('chrome', {'executable_path': 'chromedriver.exe'})

## NASA Mars News

In [4]:
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
html = browser.html
soup = bs(html, 'html')

In [5]:
content_titles = soup.find_all('div', class_ = 'content_title')
article_title = content_titles[1].text
article_title

"The Extraordinary Sample-Gathering System of NASA's Perseverance Mars Rover"

In [6]:
teaser_body = soup.find('div', class_ = 'article_teaser_body').text
teaser_body

'Two astronauts collected Moon rocks on Apollo 11. It will take three robotic systems working together to gather up the first Mars rock samples for return to Earth.'

## JPL Mars Space Images - Featured Image

In [7]:
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [8]:
browser.click_link_by_partial_text('FULL')

In [9]:
browser.click_link_by_partial_text('more info')
html = browser.html
soup = bs(html,'html')

In [10]:
lede = soup.find('figure', class_='lede')
img = lede.find('a')['href']
img_url = f'https://jpl.nasa.gov{img}'
img_url

'https://jpl.nasa.gov/spaceimages/images/largesize/PIA01486_hires.jpg'

## Mars Weather

In [11]:
from twitter import api_key, api_secret_key

In [12]:
auth = tweepy.OAuthHandler(api_key, api_secret_key)
api = tweepy.API(auth)

In [13]:
username = 'MarsWxReport'
tweets = []
data = api.user_timeline(id=username, tweet_mode="extended")

for t in data:
    tweets.append(t.full_text)
weather = tweets[0]
weather = weather.split(' http',1)[0]
weather

'InSight sol 541 (2020-06-04) low -91.8ºC (-133.2ºF) high -1.6ºC (29.2ºF)\nwinds from the SW at 5.4 m/s (12.2 mph) gusting to 17.7 m/s (39.6 mph)\npressure at 7.30 hPa'

## Mars Facts

In [14]:
url = 'https://space-facts.com/mars/'
browser.visit(url)
html = browser.html
soup = bs(html, 'html')

In [15]:
facts = pd.read_html(url)
facts_df = pd.DataFrame(facts[0])
facts_df.columns = ['Data','Value']
facts_df = facts_df.set_index('Data')
facts_df.to_html('mars_facts.html')

## Mars Hemispheres

In [16]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)
html = browser.html
soup = bs(html, 'html')

In [17]:
list_hem = []
hemispheres = {}
products = soup('div', class_='description')

for p in products:
    hem = p.find('h3').text
    hem = hem.replace(' Enhanced','')
    hemispheres['Hemisphere'] = hem
    browser.click_link_by_partial_text(hem)
    html = browser.html
    soup = bs(html,'html')
    image_url = soup.find_all('a', {'target':'_blank'})
    for i in image_url:
        if 'Sample' in i.text:
            hemispheres['Img link'] = i['href']
    list_hem.append(hemispheres)
list_hem

[{'Hemisphere': 'Valles Marineris Hemisphere',
  'Img link': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_unenhanced.tif/full.jpg'},
 {'Hemisphere': 'Valles Marineris Hemisphere',
  'Img link': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_unenhanced.tif/full.jpg'},
 {'Hemisphere': 'Valles Marineris Hemisphere',
  'Img link': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_unenhanced.tif/full.jpg'},
 {'Hemisphere': 'Valles Marineris Hemisphere',
  'Img link': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_unenhanced.tif/full.jpg'}]